In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Hypothesis Testing


Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    f = open('university_towns.txt', 'r').read()
    citystates = []
    currstate = ''
    for line in f.splitlines():
        if '[ed' in line:
            state = line.split('[')[0].strip()
        else:
            city = line.split("(")[0].strip()
            citystates.append([state, city])
    df = pd.DataFrame(citystates, columns = ["State", "RegionName"])
    return df.copy()

In [4]:
def get_housing_df():
    """ NOT FOR EVALUATION """
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    return df.copy()


In [5]:
def get_gdp():
    """ NOT FOR EVALUATION """
    df = pd.read_excel('gdplev.xls')
    df = df[[df.columns[4], df.columns[6]]]
    df.columns = ['Quarter', 'GDP']
    ind = df[df['Quarter'] == '2000q1'].index[0]
    df = df[df.index >= ind]
    df.reset_index(drop=True, inplace=True)
    return df.copy()

In [6]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = get_gdp()
    #gdp['Next'] = gdp['GDP'].shift(-1)
    #gdp['2Next'] = gdp["GDP"].shift(-2)
    gdp["RecessionStart"] = ((gdp['GDP'] < gdp['GDP'].shift(1)) & 
                             (gdp["GDP"] > gdp["GDP"].shift(-1)))
    return gdp[gdp['RecessionStart']].iloc[0]["Quarter"]


In [7]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = get_gdp()
    RecessionEnd = ((gdp["GDP"] > gdp["GDP"].shift(1)) &
                           (gdp["GDP"].shift(1) > gdp["GDP"].shift(2)))
    return gdp[RecessionEnd & (gdp["Quarter"] > get_recession_start())].iloc[0]["Quarter"]

In [8]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = get_gdp()
    rec = gdp[(get_recession_start() <= gdp["Quarter"]) & (gdp["Quarter"] <= get_recession_end())]
    return gdp.iloc[rec["GDP"].argmin()].loc["Quarter"]

In [9]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing = get_housing_df()
    columns_to_keep = ['20{:02d}-{:02d}'.format(i, j) for i in range(17) for j in range(1,13)]
    columns_to_keep = columns_to_keep[:columns_to_keep.index('2016-08')+1]
    housing["State"] = housing["State"].apply(func=lambda x: states[x])
    housing.set_index(['State', "RegionName"], inplace=True)
    housing = housing[columns_to_keep]
    housing.columns = housing.columns.to_datetime()
    housing = housing.resample('Q', axis=1).mean()
    housing.columns = ['20{:02d}q{}'.format(i, j) for i in range(17) for j in range(1,5)][:-1]
    return housing.copy()


In [19]:
def merge_housing_and_university_dfs():
    housing = convert_housing_data_to_quarters()
    uni = get_list_of_university_towns()
    uni['isunitown'] = True
    start, bottom, end = get_recession_start(), get_recession_bottom(), get_recession_end()
    housing["Drop"] = housing.iloc[:, housing.columns.get_loc(start)-1].div(housing[bottom])
    merged = pd.merge(housing, uni, left_index=True, right_on=["State", "RegionName"], how="left")
    merged['isunitown'].fillna(value=False, inplace=True)
    merged.set_index(['State', 'RegionName'], inplace=True)
    return merged.copy()

#housing[[start, bottom, 'Drop']]
#start, bottom, end = get_recession_start(), get_recession_bottom(), get_recession_end()
#housing.columns[housing.columns.slice_indexer(start=start, end=bottom)]
#housing.columns[start:bottom]
#housing.ix[:, start:bottom]
#decrease = hou

In [28]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    housing = merge_housing_and_university_dfs()
    uni_drop = housing[housing['isunitown']]['Drop']
    non_uni_drop = housing[~housing['isunitown']]['Drop']
    ttest = ttest_ind(uni_drop, non_uni_drop, nan_policy="omit")
    different = ttest.pvalue < 0.01
    p = ttest.pvalue
    better = "non-"*(ttest.statistic > 0) +'university town'
    return different, p, better


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:22: DeprecationWarning: using a boolean instead of an integer will result in an error in the future


(True, 0.0027240637047531249, 'university town')